In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm, trange
from math import sqrt
import pandas as pd
import numpy as np
import heapq
import random

In [2]:
n,m=943,1682
cols = ['user_id', 'item_id', 'rating', 'ts']
base = pd.read_csv('../../ml-100k/u1.base',sep='\t',names=cols)
test = pd.read_csv('../../ml-100k/u1.test',sep='\t',names=cols)

In [3]:
# 创建uesr-item矩阵
train_matrix = np.zeros((n, m))
train_matrix2 = np.zeros((n, m))
test_matrix = np.zeros((n, m))
y_ui = np.zeros((n, m))
TMP = np.array(base)
co = 0
total = np.array(TMP).shape[0]
while co <= total/2:
    tmp = random.randint(0, np.array(TMP).shape[0])
    try:
        train_matrix[int(TMP[tmp][0]) - 1, int(TMP[tmp][1]) - 1] = float(TMP[tmp][2])
        TMP = np.delete(TMP,[tmp],axis=0)
        co += 1
    except:
        continue
for i in TMP:
    train_matrix2[int(i[0]) - 1, int(i[1]) - 1] = float(i[2])

for i in test.itertuples():
    test_matrix[i[1] - 1, i[2] - 1] = i[3]

# 创建y矩阵
for i in range(n):
    for j in range(m):
        if train_matrix[i][j] != 0:
            y_ui[i][j] = 1

In [4]:
# Svdpp
# init
global_average_rating = train_matrix.sum()/y_ui.sum() 
avg_rating_user = np.array([0.0]*n)
avg_rating_item = np.array([0.0]*m)
for i in range(n):
    if y_ui[i].sum()==0:
        avg_rating_user[i] = global_average_rating
    else:
        avg_rating_user[i] = train_matrix[i].sum()/y_ui[i].sum()
for i in range(m):
    if y_ui[:,i].sum()==0:
        avg_rating_item[i] = global_average_rating
    else:
        avg_rating_item[i] = train_matrix[:,i].sum()/y_ui[:,i].sum()
bias_u = np.array([0.0]*n)  
bias_i = np.array([0.0]*m)
for i in range(n):
    if sum(y_ui[i]) == 0:
        bias_u[i] = 0
    else:
        t = 0
        for j in range(m):
            t += (train_matrix[i,j]-avg_rating_item[j])*y_ui[i,j]
        bias_u[i] = t/sum(y_ui[i])
for i in range(m):
    if sum(y_ui[:,i]) == 0:
        bias_i[i] = 0
    else:
        t = 0
        for j in range(n):
            t += (train_matrix[j,i]-avg_rating_user[j])*y_ui[j,i]
        bias_i[i] = t/sum(y_ui[:,i])

In [ ]:
bias_u

In [5]:
def svdpp(R, I, U, V, W, T, d, bias_u, bias_i, global_avg_rating, alpha_u=0.01, alpha_v=0.01,alpha_w=0.01,bet_u=0.01, bet_v=0.01, gama=0.01):
    minf = 9999999
    Uc = U.copy()
    Vc = V.copy()
    Wc = W.copy()
    bias_uc = bias_u.copy()
    bias_ic = bias_i.copy()
    miu = global_avg_rating
    for step in tqdm(range(T)):
        fui = 0
        Ut = np.zeros(U.shape)
        Vt = np.zeros(V.shape)
        Wt = np.zeros(W.shape)
        bias_ut = np.zeros(bias_u.shape)
        bias_it = np.zeros(bias_i.shape)
        gar = 0
        count = 0
        countU = np.zeros(bias_u.shape)
        countV = np.zeros(bias_i.shape)
        for u in range(n):
            df = pd.DataFrame(I[u])
            Iu_len = len(df[df > 0].dropna().index)
            df = W[df[df > 0].dropna().index]
            ls = []
            lt = []
            for j in range(20):
                tmp = pd.DataFrame(df).iloc[:,j]
                ls.append(float(tmp.sum()))
                lt.append(float(np.dot(tmp,tmp)))
            Uu = np.array(ls)/sqrt(Iu_len)
            Uwi = sum(lt)
            for i in range(m):
                if R[u][i] > 0:
                    count += 1
                    countU[u] += 1
                    countV[i] += 1
                    rui = np.dot(U[u,:],V[i,:].T)+np.dot(Uu,V[i,:].T)+bias_u[u]+bias_i[i]+global_avg_rating
                    if rui > 5:
                        rui = 5
                    elif rui < 1:
                        rui = 1
                    eui = R[u][i] - rui
                    fui += (np.power(eui, 2)+alpha_u*np.dot(U[u,:], U[u,:].T)+ alpha_v*np.dot(V[i,:], V[i,:].T)+bet_u*np.power(bias_u[u],2)+bet_v*np.power(bias_i[i],2)+alpha_w*Uwi)/2
                    gar -= eui
                    bias_ut[u] -= (eui-bet_u* bias_u[u])####
                    bias_it[i] -= (eui-bet_v* bias_i[i])####
                    Ut[u,:] -= (eui*V[i,:]-alpha_u*U[u,:])
                    Vt[i,:] -= (eui*(U[u,:]+Uu)-alpha_v*V[i,:])
                    Wt[i,:] -= (eui*V[i,:]/sqrt(Iu_len)-alpha_w*Wt[i,:])
        global_avg_rating -= gama*gar/count
        for u in range(n):
            if countU[u] != 0:
                bias_u[u] -= gama*bias_ut[u]/countU[u]
                U[u,:] -= gama*Ut[u,:]/countU[u]
        for i in range(m):
            if countV[i] != 0:
                bias_i[i] -= gama*bias_it[i]/countV[i]
                V[i,:] -= gama*Vt[i,:]/countV[i]
                W[i,:] -= gama*Wt[i,:]/countV [i]
        if minf > fui:
            minf = fui
            Uc = U.copy()
            Vc = V.copy()
            Wc = W.copy()
            bias_uc = bias_u.copy()
            bias_ic = bias_i.copy()
            miu = global_avg_rating
        gama = gama*0.9
    return Uc,Vc,Wc,bias_uc,bias_ic,miu   

In [6]:
# 评估函数
def eva(prediction, truth):
    prediction = prediction.flatten()
    truth = truth[truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, truth)),mean_absolute_error(prediction, truth)

In [7]:
if __name__ == '__main__':
    d = 20
    T = 100
    U = (np.random.rand(n, d)-0.5)*0.01
    V = (np.random.rand(m, d)-0.5)*0.01
    W = (np.random.rand(m, d)-0.5)*0.01
    R = train_matrix.copy()
    I = train_matrix2.copy()
    Uf, Vf, Wf,bias_uf,bias_if,miuf = svdpp(R, I, U, V, W, T, d, bias_u, bias_i, global_average_rating)
    SVDpp = np.ones((n,m))
    for i in range(n):
        for j in range(m):
            SVDpp[i,j] = np.dot(Uf[i,:],Vf[j,:].T)+np.dot(Wf[i,:],Vf[j,:].T)+bias_uf[i]+bias_if[j]+miuf
            if SVDpp[i,j] > 5:
                SVDpp[i,j] = 5
            if SVDpp[i,j] < 1:
                SVDpp[i,j] = 1

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [19:58<00:00, 11.98s/it]


In [8]:
print("SVDpp RMSE、MAE:"+str(eva(SVDpp, test_matrix)))

SVDpp RMSE、MAE:(0.9809704897937611, 0.7738195907629318)
